In [1]:
from hydromt_wflow import WflowModel
import hydromt
import os

import hydromt_rur # Custom script hydromt_rur.py that I used, can maybe be replaced by methods in the latest hydromt and hydromt_wflow version

c:\Users\hartgrin\Miniconda3\envs\Wflow_Rur\lib\site-packages\hydromt_wflow\wflow.py:10: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
ROOT = 'c:/Users/hartgrin/OneDrive - Stichting Deltares/interreg-meuse/data/catchment_rur'
BASE = 'model_a'
RUR_LIB = 'rur_thesis.yml'
DELTARES_LIB = 'p:/wflow_global/hydromt/deltares_data.yml'

os.chdir(ROOT)

### Model A: base model + forcings

Model is build using settings:

`hydromt build wflow "./models/model_a" -r "{'subbasin': [5.988550,51.185190], 'bounds': [5.9650, 50.4017, 6.7042, 51.1983], 'strord': 8}" -i "./configs/model_base.ini" --dd -vvv`

In [4]:
model_a = WflowModel(BASE, 'r+', data_libs=[RUR_LIB], deltares_data=True)

### Model B: reservoirs

Reservoirs are currently implemented as a lake, as they support discharge based on waterlevel (from the old HBV-module). Therefore, the lakes are modelled as a prismatic lake with area A and the storage func 1: S = A*H. As of latest Wflow version (PR#250), overflow is included for lakes and based on maximum storage derived from the storage func, so S_max = A * H_max. Three reservoirs are included in the model: the Rurtalsperre (combined with the Urfttalsperre and Vortalsperre as 1 big reservoir), the Oleftalsperre and the Wehebachtalsperre. The Wehebachtalsperre is difficult to calibrate as it lies in the zone affected by lignite mining and has very low outflows (0.1 m3/s < Q < 0.2 m3/s>).

In [5]:
model_b = model_a
model_b.set_root('Rur_Wflow/models/model_b', 'r+')
model_b.setup_lakes(lakes_fn='gsk3e_reservoirs', min_area=0.0)
model_b.write()
model_b.write_staticgeoms()

### Model C: reservoirs + local datasets

Based on the added map for KsatVer, the KsatHorFrac needs little calibration and is best between 500 and 1000. Using soilmaps for KsatVer results in a wide variation for KsatHorFrac between 250 to 1500. Furthermore, cross-sectional data from a hydrodynamic model (ProMaIDes) is read and mapped to the Rur river to improve performance when using the local-interial method for river routing.

In [6]:
model_c = model_b
model_c.set_root('model_c', 'r+')

REGIONS = {
        "region5": {"ID": 5, "KsatHorFrac": 500},
        "region6": {"ID": 6, "KsatHorFrac": 500}}
PATH_REGIONS = 'geo/rur_thickness.gpkg'        
regions = [REGIONS[region] for region in REGIONS.keys()]
hydromt_rur.set_usinggeom(model_c, PATH_REGIONS, "KsatHorFrac", 1000, "ID", "epsg:4326", *regions)

model_c.setup_rivers(hydrography_fn='merit_hydro', river_geom_fn='gsk3e_rivers', river_upa=5)
RIVER_RUR = "waterbodies/promaides_rur.gpkg"
hydromt_rur.set_river(model_c, 'wflow_riverwidth', RIVER_RUR, 'width', 'EPSG:31466')
hydromt_rur.set_river(model_c, 'RiverDepth', RIVER_RUR, 'depth', 'EPSG:31466', replace=10)

PATH_KSATVER = 'geo/BK50.shp'
hydromt_rur.set_vectorfile(model_c, PATH_KSATVER, "KsatVer", "kf", "epsg:25832" , scale_factor = 10)

model_c.write()
model_c.write_staticgeoms()

Running set_usinggeom for KsatHorFrac for 2 regions
New map for KsatHorFrac added
Running set_vectorfile for KsatVer
New map for KsatVer added


### Model D: reservoirs + local datasets + leakage

Lignite mining lowers the groundwater table and strongly affects the water balance. A leakage parameter is applied based on a shapefile derived from the 'Rurscholle' groundwater model, which itself is based on geological features of the area. Two areas are identified: one with higher leakage and one with lower leakage. On average, these settings result in a yearly leakage of 300-350 Mm3, which is of the same order of magnitude as the 250 Mm3 reported by the mining company.

In [7]:
model_d = model_c
model_d.set_root('Rur_Wflow/models/model_d', 'r+')

GW_leak = {
        "area_1": {"id": 1, f"MaxLeakage": 0.4},
        "area_2": {"id": 2, f"MaxLeakage": 0.8}}
PATH_GW = "geo/rurscholle.gpkg"
gw_leak = [GW_leak[gw] for gw in GW_leak.keys()]
hydromt_rur.create_usinggeom(model_d, PATH_GW, f"MaxLeakage", 0.0, "id", "epsg:4326", *gw_leak)

model_d.write()
model_d.write_staticgeoms()

Running set_usinggeom for MaxLeakage for 1 regions
New map for MaxLeakage added
